In [5]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow import keras
import matplotlib.pyplot as plt
from tensorflow.keras.layers import MaxPooling2D, GlobalAveragePooling2D, Conv2D, Dense
from tensorflow.keras import layers
import keras.backend as K
import sys
from tensorflow.keras.preprocessing.image import ImageDataGenerator

np.set_printoptions(threshold=sys.maxsize)
gpus = tf.config.experimental.list_physical_devices('XLA_GPU')
tf.config.experimental.set_visible_devices(gpus[0], 'XLA_GPU')

In [26]:
model = keras.applications.ResNet152V2(include_top=False, weights='imagenet',input_shape=(331, 331, 3))

In [27]:
model.summary()

Model: "resnet152v2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 331, 331, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 337, 337, 3)  0           input_3[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 166, 166, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
pool1_pad (ZeroPadding2D)       (None, 168, 168, 64) 0           conv1_conv[0][0]                 
________________________________________________________________________________________

# Data load

In [4]:
train = np.load('./data/pre_ok_data/train.npy')
valid = np.load('./data/pre_ok_data/valid.npy')
t_label = np.load('./data/pre_ok_data/t_label.npy')
v_label = np.load('./data/pre_ok_data/v_label.npy')

In [5]:
print('train_shape : ', train.shape, 'label_shape : ', t_label.shape)
print('valid_shape : ', valid.shape, 'label_shape : ', v_label.shape)

train_shape :  (28474, 331, 331, 3) label_shape :  (28474,)
valid_shape :  (3557, 331, 331, 3) label_shape :  (3557,)


In [6]:
from tensorflow.keras.utils import to_categorical

X_train = train.astype('float32') / 255
X_valid = valid.astype('float32') / 255

y_train = to_categorical(t_label)
y_valid = to_categorical(v_label)

print(y_train.shape, y_valid.shape)

(28474, 7) (3557, 7)


In [7]:
print('X_train : ', X_train.shape, 
     'y_train : ', y_train.shape,
     'X_valid : ', X_valid.shape,
     'y_valid : ', y_valid.shape)

X_train :  (28474, 331, 331, 3) y_train :  (28474, 7) X_valid :  (3557, 331, 331, 3) y_valid :  (3557, 7)


In [8]:
from sklearn.utils import class_weight


y_train_ = np.argmax(y_train, axis=1)
weights = class_weight.compute_class_weight('balanced',
                                            np.unique(y_train_),
                                            y_train_)

weights = {i : weights[i] for i in range(7)}

weights

c:\programdata\anaconda3\envs\testpy36\lib\site-packages\sklearn\utils\validation.py:72: FutureWarning: Pass classes=[0 1 2 3 4 5 6], y=[2 0 3 4 1 3 1 0 5 1 4 1 4 1 5 3 0 0 3 4 1 1 3 0 3 2 0 4 2 4 0 6 0 2 1 3 6
 6 0 1 4 0 0 6 2 1 3 2 1 3 1 6 0 0 1 3 2 6 0 4 0 6 0 6 1 3 0 6 1 2 6 0 4 2
 5 6 2 1 0 1 4 4 3 3 0 0 1 2 4 5 2 1 1 6 3 1 4 1 0 2 0 5 2 2 1 0 1 2 5 4 5
 1 2 6 6 4 1 0 4 1 1 4 2 0 4 0 4 2 4 0 1 0 4 0 1 2 2 2 1 3 1 0 2 2 2 1 4 4
 0 0 2 6 0 0 0 2 1 2 4 0 1 2 0 5 1 4 0 2 3 4 6 4 6 0 0 4 2 4 3 3 1 3 4 1 3
 1 0 4 6 2 4 4 1 1 0 3 1 1 3 3 2 1 3 6 4 3 0 4 3 1 0 5 4 3 0 3 2 1 2 0 1 4
 0 0 3 4 6 0 4 1 3 6 3 4 4 2 5 3 1 1 1 4 4 1 4 0 3 3 4 0 0 1 0 1 1 1 6 1 1
 2 6 5 1 1 4 2 3 1 1 3 0 1 3 2 4 5 1 1 2 3 4 0 4 1 6 0 1 0 1 2 2 4 1 2 1 4
 6 2 1 0 1 3 1 4 6 0 3 4 4 4 2 2 1 6 2 1 4 4 6 2 1 4 5 6 2 2 0 2 1 5 2 4 0
 4 0 5 3 4 5 5 6 4 0 4 5 4 1 1 4 3 4 0 2 2 4 1 6 1 3 0 0 3 0 4 3 1 4 4 1 4
 0 0 2 4 4 2 4 1 0 4 1 1 4 1 2 1 1 0 4 1 1 0 4 1 6 2 5 1 0 2 1 6 0 5 0 2 2
 2 1 1 4 1 2 1 4 1 0 0 1 0 1 0 4 4 1 4 

{0: 0.7985304840428515,
 1: 0.5646466248909336,
 2: 1.010863391082079,
 3: 1.3155608944742192,
 4: 0.8115950290730818,
 5: 2.2699298469387754,
 6: 1.80306484295846}

In [28]:
model.trainable = False

In [29]:
model.summary()

Model: "resnet152v2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 331, 331, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 337, 337, 3)  0           input_3[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 166, 166, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
pool1_pad (ZeroPadding2D)       (None, 168, 168, 64) 0           conv1_conv[0][0]                 
________________________________________________________________________________________

In [33]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
prediction_layer = tf.keras.layers.Dense(7, activation = 'softmax')

models = tf.keras.Sequential([
    model,
    global_average_layer,
    prediction_layer
])

In [34]:
models.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet152v2 (Functional)     (None, 11, 11, 2048)      58331648  
_________________________________________________________________
global_average_pooling2d_3 ( (None, 2048)              0         
_________________________________________________________________
dense_6 (Dense)              (None, 7)                 14343     
Total params: 58,345,991
Trainable params: 14,343
Non-trainable params: 58,331,648
_________________________________________________________________


In [35]:
models.compile(optimizer=tf.keras.optimizers.Adam(decay = 0.00001),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])

early_stopping = tf.keras.callbacks.EarlyStopping(patience=5)

checkpoint_path = "./b_model"

check_p = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path, save_best_only=True)

In [36]:
history = models.fit(X_train, y_train,
                    epochs=100,
                    validation_data=(X_valid, y_valid),
                    batch_size = 16,
                    class_weight=weights,
                    callbacks=[early_stopping, check_p])

Epoch 1/100
1780/1780 [==============================] - 478s 269ms/step - loss: 1.0899 - accuracy: 0.5900 - val_loss: 0.8972 - val_accuracy: 0.6550
Epoch 2/100
1780/1780 [==============================] - 490s 275ms/step - loss: 0.8312 - accuracy: 0.6826 - val_loss: 0.8562 - val_accuracy: 0.6688
Epoch 3/100
1780/1780 [==============================] - 493s 277ms/step - loss: 0.7518 - accuracy: 0.7108 - val_loss: 0.7999 - val_accuracy: 0.6877
Epoch 4/100
1780/1780 [==============================] - 481s 270ms/step - loss: 0.6982 - accuracy: 0.7292 - val_loss: 0.7237 - val_accuracy: 0.7177
Epoch 5/100
1780/1780 [==============================] - 429s 241ms/step - loss: 0.6633 - accuracy: 0.7426 - val_loss: 0.7453 - val_accuracy: 0.7130
Epoch 6/100
1780/1780 [==============================] - 430s 242ms/step - loss: 0.6357 - accuracy: 0.7538 - val_loss: 0.7285 - val_accuracy: 0.7242
Epoch 7/100
1780/1780 [==============================] - 428s 240ms/step - loss: 0.6173 - accuracy: 0.7567

In [6]:
models = keras.models.load_model('b_model/')

KeyboardInterrupt: 

In [ ]:
models.summary()

In [7]:
X_test = np.load('./data/pre_ok_data/test.npy')
y_test = np.load('./data/pre_ok_data/s_label.npy')

In [ ]:
print('X_test_shape : ', X_test.shape, 'y_test : ', y_test.shape)

In [ ]:
X_test = X_test.astype('float32') / 255
y_test = to_categorical(y_test)

In [ ]:
test_loss, test_acc = models.evaluate(X_test, y_test)

In [ ]:
print('test_loss : ', test_loss, 'test_acc : ', test_acc)

In [ ]:
prediction = models.predict(X_test)

In [ ]:
prediction_max = np.argmax(prediction, axis = 1)

In [ ]:
real_label = np.argmax(y_test, axis = 1)

In [ ]:
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix

cf = confusion_matrix(real_label, prediction_max)

classes = ['1', '2', '3', '4', '5', '6', '7']

for i in range(len(cf)):
    print(classes[i], ' class confusion matrix : ', cf[i])

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from itertools import cycle

from sklearn import svm, datasets
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from scipy import interp


y_score = models.predict(X_test)
fpr = dict()
tpr = dict()
roc_auc = dict()


for i in range(7):
    fpr[i], tpr[i], _ = roc_curve(y_test[:, i], y_score[:, i])
    roc_auc[i] = roc_auc_score(y_test[:, i], y_score[:, i])

fpr["micro"], tpr["micro"], _ = roc_curve(y_test.ravel(), y_score.ravel())
roc_auc["micro"] = roc_auc_score(y_test.ravel(), y_score.ravel())


# Plot all ROC curves
colors = cycle(['aqua', 'darkorange', 'cornflowerblue'])
for i, color in zip(range(7), colors):
    plt.plot(fpr[i], tpr[i],
             color=color, label='ROC curve of class {0} (area = {1:0.2f})'.format(i, roc_auc[i]))
plt.plot([0, 1], [0, 1],
         color='navy', linestyle='--')


plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Some extension of Receiver operating characteristic to multi-class')
plt.legend(loc="lower right")
plt.show()

In [63]:
models.save('ResNext_models/')

INFO:tensorflow:Assets written to: ResNext_models/assets
